In [15]:
'''
READ ME:
This code can pull the annual 30-year normals .bil files for precipitation.
Additionally, the code is also able to export convert the data into a space time cube and export the output animation.
The code that converts the rasters from .bil to .tif was written by Chat GPT.
'''

import requests

base_url = "https://ftp.prism.oregonstate.edu/"
api_endpoint = "normals_4km/ppt/PRISM_ppt_30yr_normal_4kmM4_all_bil.zip" # Change the API Endpoint Accordingly"
url = f'{base_url}{api_endpoint}'
local_filename = r'C:\School\GIS 5571\Local\Lab2 Data\bils.zip'
response = requests.get(url)

with open(local_filename, 'wb') as f:
    f.write(response.content)

In [16]:
import zipfile

extract_dir = r'C:\School\GIS 5571\Local\Lab2 Data\bils' # Output location
with zipfile.ZipFile(local_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [2]:
import os

folder_path = r'C:\School\GIS 5571\Local\Lab2 Data\bils'
file_list = os.listdir(folder_path)

for filename in file_list:
    if "annual" in filename.lower():  # Convert to lowercase for case-insensitive matching
        file_path = os.path.join(folder_path, filename)
        try:
            os.remove(file_path)
            print(f"Deleted: {filename}")
        except Exception as e:
            print(f"Error deleting {filename}: {e}")

In [18]:
import arcpy

bils_folder = r'C:\School\GIS 5571\Local\Lab2 Data\bils'
tifs_folder = r'C:\School\GIS 5571\Local\Lab2 Data\tifs'
bil_files = [f for f in os.listdir(bils_folder) if f.lower().endswith('.bil')]


for bil_file in bil_files:
    bil_path = os.path.join(bils_folder, bil_file)
    tif_file = os.path.splitext(bil_file)[0] + '.tif'
    tif_path = os.path.join(tifs_folder, tif_file)
    if not os.path.exists(tif_path):
        arcpy.conversion.RasterToOtherFormat(bil_path, tifs_folder, "TIFF")

In [1]:
coordinate_system = arcpy.SpatialReference(4269)  # NAD 1983
in_workspace = r'C:\School\GIS 5571\Local\Lab2 Data\Lab2 Project\Lab2 Project.gdb'
in_mosaicdataset_name = 'Normals_Precipiation'
arcpy.management.CreateMosaicDataset(in_workspace, in_mosaicdataset_name, coordinate_system)

<Result 'C:\\School\\GIS 5571\\Local\\Lab2 Data\\Lab2 Project\\Lab2 Project.gdb\\Normals_Precipiation'>

In [2]:
in_mosaic_dataset = 'Normals_Precipiation'
input_path = r'C:\School\GIS 5571\Local\Lab2 Data\tifs'
raster_type = 'Raster Dataset'
arcpy.management.AddRastersToMosaicDataset(in_mosaic_dataset, raster_type, input_path)

<Result 'Normals_Precipiation'>

In [3]:
field_name = 'Variable'
expression = '"ppt"'
arcpy.CalculateField_management(in_mosaic_dataset, field_name, expression, "PYTHON3")

<Result 'Normals_Precipiation'>

In [11]:
field_name = 'Timestamp'
type_field = 'DATE'
expression = 'DateAdd(Date( 1990, 12, 1), $feature.OBJECTID - 1, "month")'
arcpy.management.CalculateField(in_mosaic_dataset, field_name, expression, "ARCADE", field_type = type_field)

<Result 'Normals_Precipiation'>

In [12]:
variable_field = 'Variable'
timestamp_field = 'Timestamp'
arcpy.md.BuildMultidimensionalInfo(in_mosaic_dataset, variable_field, timestamp_field)

<Result 'Normals_Precipiation'>

In [13]:
in_multidimensional_raster = in_mosaic_dataset
out_multidimensional_raster_layer = 'Normals_Precipitation_MultiDim_Layer'
arcpy.md.MakeMultidimensionalRasterLayer(in_multidimensional_raster, out_multidimensional_raster_layer)

<Result 'Normals_Precipitation_MultiDim_Layer'>

In [19]:
output_cube = r'C:\School\GIS 5571\Local\Lab2 Data\STCube\ppt.nc'
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(out_multidimensional_raster_layer, output_cube, 'Zeros')

<Result 'C:\\School\\GIS 5571\\Local\\Lab2 Data\\STCube\\ppt.nc'>